# Importar Archivos DE titulos y areas de interes


In [ ]:
# Import Data
import pandas as pd
import numpy as np
df_titles = pd.read_excel('/content/titles_scholar_profile_keywords.xlsx')
df_areas = pd.read_excel('/content/interests_scholar_profile.xlsx')

In [ ]:
#Merge data
df_raw = pd.merge(df_titles, df_areas, on='Researcher')

# Quit column Paper Titles
df_raw_v2 = df_raw.drop(columns=['Paper Titles'])
df_raw_v2

,Researcher,keywords,Interest Areas
0,Omar Caceres,"Prueba molecular, Genotype® MTBDRplus, tubercu...","Molecular Biology, Genomics, Bioinformatics, P..."
1,Omar Caceres,"Evolutionary, genomic dynamics, Peruvians","Molecular Biology, Genomics, Bioinformatics, P..."
2,Omar Caceres,"molecular characterization, Echinococcus granu...","Molecular Biology, Genomics, Bioinformatics, P..."
3,Omar Caceres,"Whole-genome sequencing, elderly, Brazil","Molecular Biology, Genomics, Bioinformatics, P..."
4,Omar Caceres,"genomic analysis, spread, lambda SARS-COV-2","Molecular Biology, Genomics, Bioinformatics, P..."
...,...,...,...
4235,Elizabeth Tapia,"Subset selection, protein attributes, Fuzzy In...","Bioinformatics, Error Correcting Codes, Networ..."
4236,Elizabeth Tapia,"power, binary learners, non-binary classificat...","Bioinformatics, Error Correcting Codes, Networ..."
4237,Elizabeth Tapia,"In Silico Analysis, Small Heat Shock Protein G...","Bioinformatics, Error Correcting Codes, Networ..."
4238,Elizabeth Tapia,"Gene Set Enrichment Analysis, Non-parametric S...","Bioinformatics, Error Correcting Codes, Networ..."


Función para clasificar los keywords de acuerdo a areas de interes

# Clasificar Keywords en base a las areas de interes

In [ ]:
!pip install openai
from openai import OpenAI

In [ ]:
client = OpenAI(api_key='')  # Replace with your actual key

def categorize_keywords(keywords, interest_area):
     chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant programmer expert on Bioinformatical and Biological topics that categorizes keywords based on given interest areas."
            },
            {
                "role": "user",
                "content" : (
                    f"Categorize the following keywords based on the interest area: {interest_area}. "
                    f"Keywords: {keywords}. Use 'Others' as an interest area if a keyword does not have any relationship with the provided interest areas. "
                    "Provide the output as a list of dictionaries, where each dictionary contains a 'keyword' and its 'interest_area'. "
                    "Ensure the output is in a consistent format, like the following example:\n"
                    "[\n"
                    "    {'keyword': 'Prueba molecular', 'interest_area': 'Molecular Biology'},\n"
                    "    {'keyword': 'Genotype® MTBDRplus', 'interest_area': 'Genomics'},\n"
                    "    {'keyword': 'multidrug-resistant tuberculosis', 'interest_area': 'Others'}\n"
                    "]\n"
                    "Also, if you detect spelling errors, please correct them. And translate all keywords and interest areas into English."
                )
             },
        ], model="gpt-3.5-turbo",
    )
     return chat_completion.choices[0].message.content

In [ ]:
#Make df_test
df_test = df_raw_v2.head(2)

# Use function to df_test
df_test['Categorized Keywords'] = df_test.apply(lambda row: categorize_keywords(row['keywords'], row['Interest Areas']), axis=1)

df_test
#export to excel
df_test.to_excel('df_test.xlsx', index=False)

<ipython-input-36-841bcae2072e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Categorized Keywords'] = df_test.apply(lambda row: categorize_keywords(row['keywords'], row['Interest Areas']), axis=1)


In [ ]:
#Separar los df ya que son muy pesados se Divide el DataFrame en 4 partes iguales [de 4240 a 4 de 1060 filas]
df1 = df_raw_v2.iloc[:1060]
df2 = df_raw_v2.iloc[1060:2120]
df3 = df_raw_v2.iloc[2120:3180]
df4 = df_raw_v2.iloc[3180:]

In [ ]:
from tqdm import tqdm
tqdm.pandas()
df4['Classified Raw Keywords'] = df4.progress_apply(lambda row: categorize_keywords(row['keywords'], row['Interest Areas']), axis=1)

# Export
df4.to_excel('df4.xlsx', index=False)

from google.colab import files

# Supongamos que tienes un archivo llamado 'archivo_a_descargar.txt'
files.download('df4.xlsx')

In [22]:
# read excels
import pandas as pd

# Leer los archivos Excel en DataFrames
df1 = pd.read_excel('/content/df1.xlsx')
df2 = pd.read_excel('/content/df2.xlsx')
df3 = pd.read_excel('/content/df3.xlsx')
df4 = pd.read_excel('/content/df4.xlsx')

In [39]:
import json
import ast
import pandas as pd

# Aplicar la función a la columna 'Classified Raw Keywords'
def clean_and_parse_dataframe(df, column):
    def parse_classified_keywords(json_str):
        # Si json_str es una lista returna lo mismo
        if isinstance(json_str, list):
            return json_str
        # Eliminar caracteres de nueva línea y espacios en blanco
        json_str = json_str.replace("\n", "").strip()

        # Limpiar prefijos de formato si existen
        if '```' in json_str:
            json_str = json_str.replace('```json', '').replace('```plaintext', '').replace('```', '').strip()

        # Intentar cargar como JSON
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            # Reemplazar comillas simples por dobles e intentar cargar de nuevo
            try:
                json_str = json_str.replace("'", '"')
                return json.loads(json_str)
            except json.JSONDecodeError:
                # Usar ast.literal_eval como último recurso
                try:
                    return ast.literal_eval(json_str)
                except (ValueError, SyntaxError):
                    # Si falla, retornar None
                    return None

    # Aplicar la función de parseo y limpiar filas con datos no válidos
    original_length = len(df)
    df[column] = df[column].apply(parse_classified_keywords)
    df_cleaned = df.dropna(subset=[column])
    removed_count = original_length - len(df_cleaned)

    return df_cleaned, removed_count

In [ ]:
df_1_test = df4

# Aplicar la función a df_1_test
df_cleaned, removed_count = clean_and_parse_dataframe(df_1_test, 'Classified Raw Keywords')

print(f"Filas eliminadas: {removed_count}")
#print(df_cleaned)

# Quitar duplicados
df_cleaned = df_cleaned.drop_duplicates(subset=['keywords'])
#df_cleaned

# Explode the list of dictionaries into separate rows
df_exploded = df_cleaned.explode('Classified Raw Keywords')

# Convertir cada diccionario en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['Classified Raw Keywords'], axis=1), df_exploded['Classified Raw Keywords'].apply(pd.Series)], axis=1)

#Export df_exploded
df_exploded.to_excel('df4_raw.xlsx', index=False)

# Create a df only with 'Others' and another with the rest
df_others = df_exploded[df_exploded['interest_area'] == 'Others']

df_non_others = df_exploded[df_exploded['interest_area'] != 'Others']

# Export df_others
df_others.to_excel('df4_others.xlsx', index=False)
df_non_others.to_excel('df4_significant_keywords.xlsx', index=False)

# Download all xlsx
from google.colab import files

files.download('df4_others.xlsx')
files.download('df4_raw.xlsx')
files.download('df4_significant_keywords.xlsx')


In [ ]:
#!pip install googletrans==3.1.0a0
#from googletrans import Translator
#!pip install deep-translator

In [49]:
from deep_translator import GoogleTranslator

def translate_keywords(text, src='es', dest='en'):
    """
    Esta función traduce un texto dado desde el idioma de origen al idioma de destino.

    Args:
    text (str): Texto a traducir.
    src (str): Código del idioma de origen.
    dest (str): Código del idioma de destino.

    Returns:
    str: Texto traducido.
    """
    try:
        translated = GoogleTranslator(source=src, target=dest).translate(text)
        return translated
    except Exception as e:
        print(f"Error translating {text}: {e}")
        return text

from tqdm import tqdm
tqdm.pandas()
# Apply the function to the 'text' column
df_non_others['English'] = df_non_others['keyword'].progress_apply(translate_keywords, src = 'es', dest = 'en')

# Apply Later when all df are already to translate in the main data
#df_non_others['Spanish'] = df_non_others['keyword'].progress_apply(translate_keywords, src = 'en', dest = 'es')

#Export
df_non_others.to_excel('df4_significant_keywords_translated_v1.xlsx', index=False)
#Download
from google.colab import files

files.download('df4_significant_keywords_translated_v1.xlsx')

100%|██████████| 1844/1844 [08:31<00:00,  3.60it/s]
<ipython-input-49-be053fd83777>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_others['English'] = df_non_others['keyword'].progress_apply(translate_keywords, src = 'es', dest = 'en')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Curar data manualmente . . .

In [57]:
# Import df1_others_curated
import pandas as pd
df_sig_curated = pd.read_excel('/content/df4_significant_keywords_curated_areas_v1.xlsx')
df_sig_curated.columns

#Drop some columns of df_sig_curated
df_sig_curated = df_sig_curated.drop(['keywords', 'Interest Areas',], axis=1)
df_sig_curated


,Researcher,keyword,interest_area,English
0,Alejandra Medina-Rivera,motif clustering,Bioinformatics,motif clustering
1,Alejandra Medina-Rivera,omics data sets,Bioinformatics,omics data sets
2,Alejandra Medina-Rivera,RSAT,Bioinformatics,RSAT
3,Alejandra Medina-Rivera,RSAT,Bioinformatics,RSAT
4,Alejandra Medina-Rivera,regulatory sequence analysis tools,Bioinformatics,regulatory sequence analysis tools
...,...,...,...,...
1834,Ruy D. Chacón,BR-I serotype,Virology,BR-I serotype
1835,Ruy D. Chacón,Chicken astrovirus,Virology,Chicken astrovirus
1836,Ruy D. Chacón,detection,Virology,detection
1837,Ruy D. Chacón,molecular characterization,Virology,molecular characterization


In [58]:
import pandas as pd

def separate_and_group_keywords(df):
    """
    Esta función toma un DataFrame con columnas 'Researcher', 'keywords' y 'interest_area'.
    Separa el DataFrame en DataFrames individuales por investigador y agrupa las palabras clave
    según las áreas de interés, separándolas por comas. Luego une todos los DataFrames en uno
    solo bajo las columnas 'Researcher', 'keywords' y 'interest_area'.

    Args:
    df (pd.DataFrame): DataFrame con las columnas 'Researcher', 'keywords' y 'interest_area'.

    Returns:
    pd.DataFrame: DataFrame resultante con las palabras clave agrupadas.
    """

    # Obtener la lista de investigadores únicos
    researchers = df['Researcher'].unique()

    # Crear una lista para almacenar los DataFrames por investigador
    grouped_dfs = []

    for researcher in researchers:
        # Filtrar el DataFrame original por investigador
        df_researcher = df[df['Researcher'] == researcher]

        # Agrupar las palabras clave por 'interest_area', separándolas por comas
        df_grouped = df_researcher.groupby(['interest_area'])['English'].apply(lambda x: ', '.join(x)).reset_index()

        # Añadir la columna 'Researcher'
        df_grouped['Researcher'] = researcher

        # Renombrar la columna resultante
        df_grouped.rename(columns={'English': 'Grouped keywords'}, inplace=True)

        # Añadir el DataFrame resultante a la lista
        grouped_dfs.append(df_grouped)

    # Concatenar todos los DataFrames en uno solo
    result_df = pd.concat(grouped_dfs, ignore_index=True)

    # Reorganizar las columnas
    result_df = result_df[['Researcher', 'interest_area', 'Grouped keywords']]

    return result_df

# Llamar a la función con el DataFrame de ejemplo
df_grouped = separate_and_group_keywords(df_sig_curated)
df_grouped

# Export
df_grouped.to_excel('df4_grouped_keywords.xlsx', index=False)
#Download
from google.colab import files

files.download('df4_grouped_keywords.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
from collections import Counter
def get_most_common_keywords(text, n=15):
    # Split the text into keywords
    words = text.split(',')
    # Count the frequencies of each word
    word_count = Counter(words)
    # Get the 'n' most common words
    most_common_words = [word for word, count in word_count.most_common(n)]
    # Join the most common words back into a single string
    return ','.join(most_common_words)

# Apply the function to each cell in the 'text' column
df_grouped['most_common_keywords'] = df_grouped['Grouped keywords'].apply(get_most_common_keywords)

# Separar las palabras clave en listas y eliminar duplicados
df_grouped['most_common_keywords'] = df_grouped['most_common_keywords'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))

#Export
df_grouped.to_excel('df4_most_common_keywords.xlsx', index=False)
#Download
from google.colab import files

files.download('df4_most_common_keywords.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
df_grouped['most_common_keywords'] = df_grouped['most_common_keywords'].str.split(', ')
df_exploded = df_grouped.explode('most_common_keywords')

#Drop one column
df_exploded = df_exploded.drop(['Grouped keywords'], axis=1)
df_exploded['most_common_keywords'] = df_exploded['most_common_keywords'].str.title()

#Export df_exploded
df_exploded.to_excel('df4_final.xlsx', index=False)

#Dowload
files.download('df4_final.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
# Import df#_FINAL.slsx
import pandas as pd
df1_final = pd.read_excel('/content/df1_FINAL.xlsx')
df2_final = pd.read_excel('/content/df2_FINAL.xlsx')
df3_final = pd.read_excel('/content/df3_FINAL.xlsx')
df4_final = pd.read_excel('/content/df4_FINAL.xlsx')

# Concat df_final:
df_final = pd.concat([df1_final, df2_final, df3_final, df4_final], ignore_index=True)
df_final

# Export
df_final.to_excel('FINAL.xlsx', index=False)
#Download
from google.colab import files

files.download('FINAL.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
# import Final and Metada_ponentes.xlsx
import pandas as pd
df_final_scholar_name = pd.read_excel('/content/FINAL.xlsx')
df_metadata = pd.read_excel('/content/Metadata_ponentes.xlsx')


df_metadata.columns

Index(['Country', 'Grade', 'Names', 'Institution', 'ORCID', 'ScholarName',
       'keywords', 'areas'],
      dtype='object')

In [78]:
df_final_scholar_name.columns

Index(['ScholarName', 'interest_area', 'most_common_keywords'], dtype='object')

In [90]:
df_metadata.columns
df_metadata['NAMES'] = df_metadata['Grade'] + ' ' + df_metadata['Names']

#Rename Researcher to ScholarNAme from df_final_scholar_name
df_final_scholar_name = df_final_scholar_name.rename(columns={'Researcher': 'ScholarName'})

# Merge Collumns from df_metadata['NAMES']  into df_scholar_name
df_final_scholar_name = pd.merge(df_final_scholar_name, df_metadata[['ScholarName', 'NAMES']], on='ScholarName', how='right')
columns = ['NAMES'] + [col for col in df_final_scholar_name.columns if col != 'NAMES']
df_final_scholar_name = df_final_scholar_name[columns]

df_final_scholar_name
#Export
df_final_scholar_name.to_excel('FINAL_real_names.xlsx', index=False)
#Download
from google.colab import files

files.download('FINAL_real_names.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Import files
import pandas as pd
df_final_names = pd.read_excel('/content/FINAL_real_names.xlsx')
df_metadata = pd.read_excel('/content/Metadata_ponentes.xlsx')
df_profiles = pd.read_excel('/content/profiles_v4.xlsx')

# Merge Scholar Profiles
df_final_names = pd.merge(df_final_names, df_profiles, on='ScholarName', how='left')

# Merge metadata
df_final = pd.merge(df_final_names, df_metadata[['Country','NAMES','ScholarName', 'Institution', 'ORCID']], on='NAMES', how='right')

columns = ['NAMES'] + [col for col in df_final.columns if col != 'NAMES']
df_final = df_final[columns]

#Export
df_final.to_excel('FINAL_v1.xlsx', index=False)
#Download
from google.colab import files

files.download('FINAL_v1.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>